In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import warnings
from sklearn.metrics import ConfusionMatrixDisplay
warnings.simplefilter(action='ignore', category=FutureWarning)

PATH_FLOD = os.path.join(os.getcwd(), '..')

sys.path.append(os.path.abspath(PATH_FLOD))

In [ ]:
MAX_ROUNDS = 1
NOISE = .1
TOL = 0.0001

In [ ]:
from flod.classifiers.dp_flbsv import DPFLBSV

In [ ]:
from experiments.experiments import get_datasets, get_dataset_from_path

In [ ]:
X, Y = get_dataset_from_path(get_datasets()['pen-global'])

In [ ]:
from sklearn.model_selection import train_test_split

# Keep only normal data for training
X_train = []
y_train = []
for x, y in zip(X, Y):
    if y == 1:
        X_train.append(x)
        y_train.append(y)


X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=0.985)
X_train = np.array(X_train)
y_train = np.array(y_train)
len(X_train)

# IID

In [ ]:
client_assigment = np.random.choice([0,1], size=len(X_train))

In [ ]:
clf = DPFLBSV(NOISE, TOL, normal_class_label=1, outlier_class_label=-1, max_rounds=MAX_ROUNDS)

In [ ]:
from sklearn.metrics import accuracy_score

acc_res = []
def callback(fbsv):
    acc = accuracy_score(y_train, fbsv.clf.predict(X_train))
    acc_res.append(acc)

    plt.plot(acc_res)
    plt.show()

In [ ]:
clf.fit(X_train, y_train, client_assignment=client_assigment, round_callback=callback)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_train, clf.predict(X_train), cmap=plt.cm.Blues)
plt.show()

# Toy Example

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler

X, client_assigment = make_blobs(n_samples=12, centers=2, n_features=2, random_state=42)
y = [1 for _ in range(len(X))]

X = MinMaxScaler().fit_transform(X)

In [ ]:
# Imbalance the clients
new_X = []
new_y = []
new_ass = []

for i, a in enumerate(client_assigment):
    if a == 0:
        keep = np.random.choice([0,1], size=1, p=[.5, .5])
        if keep[0] == 1:
            continue
    new_X.append(X[i])
    new_y.append(y[i])
    new_ass.append(client_assigment[i])

X = np.array(new_X)
y = np.array(new_y)
client_assigment = new_ass

In [ ]:
C = 1
q = 1

In [ ]:
plt.scatter(X[:,0], X[:,1], c=client_assigment)
plt.show()

In [ ]:
acc_res = []
def callback(fbsv):
    acc = accuracy_score(y, fbsv.clf.predict(X))
    acc_res.append(acc)

    plt.plot(acc_res)
    plt.show()

    gx = np.linspace(min(X[:,0])-1, max(X[:,0])+1)
    gy = np.linspace(min(X[:,1])-1, max(X[:,1])+1)
    gX, gY = np.meshgrid(gx, gy)
    zs = np.array([clf.clf._compute_r((x,y)) for x, y in zip(np.ravel(gX), np.ravel(gY))])
    gZ = zs.reshape(gX.shape)
    levels = [clf.clf._best_radius()]
    levels.sort()
    membership_contour = plt.contour(gX, gY, gZ, levels=levels)
    plt.clabel(membership_contour, inline=1)

    plt.scatter(X[:, 0], X[:, 1], c=client_assigment, alpha=.4)

    plt.show()

In [ ]:
clf = DPFLBSV(NOISE, TOL, normal_class_label=1, outlier_class_label=-1, max_rounds=MAX_ROUNDS, C = C, q = q)

In [ ]:
clf.fit(X, y, client_assignment=client_assigment, round_callback=callback)

In [ ]:
gx = np.linspace(min(X[:,0])-1, max(X[:,0])+1)
gy = np.linspace(min(X[:,1])-1, max(X[:,1])+1)
gX, gY = np.meshgrid(gx, gy)
zs = np.array([clf.clf._compute_r((x,y)) for x, y in zip(np.ravel(gX), np.ravel(gY))])
gZ = zs.reshape(gX.shape)
levels = [clf.clf._best_radius()]
levels.sort()
membership_contour = plt.contour(gX, gY, gZ, levels=levels)
plt.clabel(membership_contour, inline=1)

plt.scatter(X[:, 0], X[:, 1], c=client_assigment, alpha=.4)

plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(y, clf.predict(X), cmap=plt.cm.Blues)
plt.show()